# **DESAFÍO Nº5: INFERENCIA E HIPÓTESIS**
### **Módulo 4: Análisis Estadístico con Python**
#### *Carrera Data Science - G-93 - Desafio Latam*
#### Estudiante: Felipe Leal Arancibia

En este desafío validaremos nuestros conocimientos de estadística inferencial. Para lograrlo, realizarás inferencias y pruebas de hipótesis a partir de la base de datos
earnings.csv.

Lee todo el documento antes de comenzar el desarrollo individual, para asegurarte de tener el máximo de puntaje y enfocar bien los esfuerzos.

Tiempo asociado: 2 horas cronológicas

## Descripción

#### Podemos registrar muchas características de una persona, y buscar ver si algunas influyen o no en otras. Realiza las siguientes actividades para verificarlo.

In [2]:
# Primero importar todas las potenciales librería sa utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


#### 1. Considerando las variables earn, height y age, vamos a suponer que nuestro dataset fuera una población completa. (Prepara los datos adecuadamente).

In [3]:

# Cargamos el dataset
df_earnings = pd.read_csv('earnings.csv')

df_earnings.shape

(1816, 15)

In [4]:
df_earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   height            1816 non-null   int64  
 1   weight            1789 non-null   float64
 2   male              1816 non-null   int64  
 3   earn              1816 non-null   float64
 4   earnk             1816 non-null   float64
 5   ethnicity         1816 non-null   object 
 6   education         1814 non-null   float64
 7   mother_education  1572 non-null   float64
 8   father_education  1521 non-null   float64
 9   walk              1816 non-null   int64  
 10  exercise          1816 non-null   int64  
 11  smokenow          1815 non-null   float64
 12  tense             1815 non-null   float64
 13  angry             1815 non-null   float64
 14  age               1816 non-null   int64  
dtypes: float64(9), int64(5), object(1)
memory usage: 212.9+ KB


In [5]:
df_earnings.tail(3)

,height,weight,male,earn,earnk,ethnicity,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
1813,72,194.0,1,60000.0,60.0,White,12.0,12.0,12.0,2,1,2.0,0.0,0.0,50
1814,63,155.0,0,15000.0,15.0,Other,14.0,14.0,14.0,6,1,2.0,2.0,2.0,69
1815,68,150.0,1,6000.0,6.0,White,12.0,12.0,12.0,1,6,1.0,2.0,2.0,27


Allí se puede ver el cuadro síntesis de nuestra  "población completa", a tener en cuenta en pasos siguientes.

Se crea una variable que contenga un filtro de las categorías del dataset: earn, height y age.

In [6]:
# Crear variable que filtre columnas earn, height y age
filtro_earnings = df_earnings[['height','earn', 'age']] 
filtro_earnings


,height,earn,age
0,74,50000.0,45
1,66,60000.0,58
2,64,30000.0,29
3,65,25000.0,57
4,63,50000.0,91
...,...,...,...
1811,61,15000.0,82
1812,64,8000.0,33
1813,72,60000.0,50
1814,63,15000.0,69


In [7]:
filtro_earnings.isna().sum()

height    0
earn      0
age       0
dtype: int64

In [8]:
# Se muestran algunos parámetros de las columnas a trabajar. Incluye mean y std.
round(filtro_earnings.describe(),1)

,height,earn,age
count,1816.0,1816.0,1816.0
mean,66.6,21147.3,42.9
std,3.8,22531.8,17.2
min,57.0,0.0,18.0
25%,64.0,6000.0,29.0
50%,66.0,16000.0,39.0
75%,69.2,27000.0,56.0
max,82.0,400000.0,91.0


##### a. Realiza 4 pruebas con muestras de tamaño n = 25, en cada caso, con niveles de significancia diferentes pero inferiores a 0,1, para confirmar o rechazar las siguientes hipótesis considerando las alternativas. Crea para ello una fórmula que reciba los parámetros adecuados y responda "con nivel de confianza del ...%, se acepta la hipótesis nula/alternativa".

In [9]:
# Paso a paso recomendado para hacer prueba de hipótesis
# Paso 1: Planteamiento de hipótesis
# Paso 2: Elección nivel de significancia
# Paso 3: Selección y cálculo de un estadístico de prueba
# Paso 4: Determinación de la región crítica - P values
# Paso 5: Toma de decisión

# Se modifican un poco 

In [10]:
# Paso 1: Planteamiento de hipótesis

In [11]:
#earn                        height                  age
#H0: x == 20000        |       H0: x == 55       |       H0: x == 44
#H1: x != 20000        |       H1: x != 55       |       H1: x != 44
# ______________________________________________________________
#H0: x == 35000        |       H0: x == 68        |      H0: x == 56
#H1: x != 35000        |       H1: x != 68        |      H1: x != 56
#_______________________________________________________________
#H0: x <= 25000        |       H0: x <= 63        |      H0: x <= 51
#H1: x > 25000        |       H1: x > 63        |      H1: x > 51
#_______________________________________________________________
#H0: x >= 32000        |       H0: x >= 70        |      H0: x >= 62
#H1: x < 32000        |       H1: x < 70        |      H1: x < 62

In [12]:
# Paso 2: Elección nivel de significancia y valores del ejercicio
n = 25
alphas = [0.03, 0.05, 0.07, 0.09]  # se hace una lista con 4 valores de significancia distinta, menores a 0.1

In [13]:
# Se traducen valores descritos y retoman las hipótesis por cada una de las variables, a modo de listas para luego iterar sobre ellas

hipotesis_earn = [
    (20000, 'two-sided', alphas[0]),
    (35000, 'two-sided', alphas[1]),
    (25000, 'greater', alphas[2]),
    (32000, 'less', alphas[3]),
]

hipotesis_height = [
    (55, 'two-sided', alphas[0]),
    (68, 'two-sided', alphas[1]),
    (63, 'greater', alphas[2]),
    (70, 'less', alphas[3]),
]

hipotesis_age = [
    (44, 'two-sided', alphas[0]),
    (56, 'two-sided', alphas[1]),
    (51, 'greater', alphas[2]),
    (62, 'less', alphas[3]),
]
 

In [46]:
# Paso 3: Selección y cálculo de un estadístico de prueba
# Paso 4: Determinación de la región crítica - P values

# Se crea una función para realizar las pruebas de hiótesis, considerando distribución t student, ya que la muestra es menor a 30

def prueba_hipotesis(df, variable, n, h0, alternative, alpha):

    # Se hace validación d eparametros
    if variable not in df.columns:
        raise ValueError(f"La variable '{variable}' no existe en el DataFrame.")
    if alternative not in ['two-sided', 'greater', 'less']:
        raise ValueError("El parámetro 'alternative' debe ser: 'two-sided', 'greater' o 'less'.")
    
    # Se toma una muestra 
    muestra = df[variable].sample(n, random_state=1)
    confianza = (1-alpha)*100
    
    # Se calcula la media de la muestra, t student y p value (valor crítico)
    t_stat, p_value = stats.ttest_1samp(muestra, h0, alternative=alternative)

    # Voy a traducir los valores de alternative a español
    traduccion_alternative = {
        'two-sided': 'diferente',
        'greater': 'mayor',
        'less': 'menor'}
    alternativa_traducida = traduccion_alternative[alternative]



    
    # Imprimir valores:
    print(f"Variable: {variable}")
    print(f"Tamaño muestral: {n}")
    print(f"H0: Media poblacional es {h0}.")
    print(f"H1: Media poblacional es {alternativa_traducida} a {h0}.")
    print(f"Nivel de significancia: {alpha}")
    print(f"Valor t: {t_stat}")
    print(f"Valor p: {p_value}")
    
    # Comprobar si el valor p es menor que alfa para rechazar la hipótesis nula
    if p_value < alpha:
        resultado = f"Con nivel de confianza de {confianza}%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa."
    else:
        resultado = f"Con nivel de confianza de {confianza}%, NO se rechaza la hipótesis nula."

    print(resultado, "\n")
    
    return {
        "t_stat": t_stat,
        "variable": variable,
        "n": n,
        "h0": h0,
        "h1": f"Media poblacional {alternativa_traducida} a {h0}",
        "alpha": alpha,
        "t_stat": t_stat,
        "p_value": p_value,
        "resultado": resultado
    }

    


In [47]:
# Paso 5: Toma de decisión

# Se procede con las 4 pruebas de columna earn
pruebas_earn = [prueba_hipotesis(filtro_earnings, 'earn', n, h0, alternative, alpha) for h0, alternative, alpha in hipotesis_earn]


Variable: earn
Tamaño muestral: 25
H0: Media poblacional es 20000.
H1: Media poblacional es diferente a 20000.
Nivel de significancia: 0.03
Valor t: -2.1380387085215293
Valor p: 0.04290057846389046
Con nivel de confianza de 97.0%, NO se rechaza la hipótesis nula. 

Variable: earn
Tamaño muestral: 25
H0: Media poblacional es 35000.
H1: Media poblacional es diferente a 35000.
Nivel de significancia: 0.05
Valor t: -8.011771424240017
Valor p: 3.074893587522929e-08
Con nivel de confianza de 95.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa. 

Variable: earn
Tamaño muestral: 25
H0: Media poblacional es 25000.
H1: Media poblacional es mayor a 25000.
Nivel de significancia: 0.07
Valor t: -4.095949613761025
Valor p: 0.9997933263436519
Con nivel de confianza de 93.0%, NO se rechaza la hipótesis nula. 

Variable: earn
Tamaño muestral: 25
H0: Media poblacional es 32000.
H1: Media poblacional es menor a 32000.
Nivel de significancia: 0.09
Valor t: -6.837024881096319
Valor p:

In [48]:
# Se procede con las 4 pruebas de columna height
pruebas_height = [prueba_hipotesis(filtro_earnings, 'height', n, h0, alternative, alpha) for h0, alternative, alpha in hipotesis_height]

Variable: height
Tamaño muestral: 25
H0: Media poblacional es 55.
H1: Media poblacional es diferente a 55.
Nivel de significancia: 0.03
Valor t: 17.64352905875293
Valor p: 3.0170593817782123e-15
Con nivel de confianza de 97.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa. 

Variable: height
Tamaño muestral: 25
H0: Media poblacional es 68.
H1: Media poblacional es diferente a 68.
Nivel de significancia: 0.05
Valor t: -1.860372111637205
Valor p: 0.07513055727534948
Con nivel de confianza de 95.0%, NO se rechaza la hipótesis nula. 

Variable: height
Tamaño muestral: 25
H0: Media poblacional es 63.
H1: Media poblacional es mayor a 63.
Nivel de significancia: 0.07
Valor t: 5.641128338512846
Valor p: 4.142944593189397e-06
Con nivel de confianza de 93.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa. 

Variable: height
Tamaño muestral: 25
H0: Media poblacional es 70.
H1: Media poblacional es menor a 70.
Nivel de significancia: 0.09
Valor t: -4.8609

In [49]:
# Se procede con las 4 pruebas de columna age
pruebas_age = [prueba_hipotesis(filtro_earnings, 'age', n, h0, alternative, alpha) for h0, alternative, alpha in hipotesis_age]

Variable: age
Tamaño muestral: 25
H0: Media poblacional es 44.
H1: Media poblacional es diferente a 44.
Nivel de significancia: 0.03
Valor t: -1.3023984977655982
Valor p: 0.2051405295742126
Con nivel de confianza de 97.0%, NO se rechaza la hipótesis nula. 

Variable: age
Tamaño muestral: 25
H0: Media poblacional es 56.
H1: Media poblacional es diferente a 56.
Nivel de significancia: 0.05
Valor t: -4.354894976903721
Valor p: 0.00021422444987887325
Con nivel de confianza de 95.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa. 

Variable: age
Tamaño muestral: 25
H0: Media poblacional es 51.
H1: Media poblacional es mayor a 51.
Nivel de significancia: 0.07
Valor t: -3.0830214439295034
Valor p: 0.9974547081913591
Con nivel de confianza de 93.0%, NO se rechaza la hipótesis nula. 

Variable: age
Tamaño muestral: 25
H0: Media poblacional es 62.
H1: Media poblacional es menor a 62.
Nivel de significancia: 0.09
Valor t: -5.881143216472783
Valor p: 2.2835783990432878e-06
Con

##### b. Calcula la media poblacional para cada variable. ¿Son correctos los resultados obtenidos por tus pruebas?

In [50]:
# Se calcula la media poblacional para cada variable
media_poblacional_earn = round(df_earnings['earn'].mean(),2)
media_poblacional_height = round(df_earnings['height'].mean(),2)
media_poblacional_age = round(df_earnings['age'].mean(),2)

# Se imprimen resultados
print(f"Media poblacional de variable 'earn': {media_poblacional_earn}")
print(f"Media poblacional de variable 'height': {media_poblacional_height}")
print(f"Media poblacional de variable 'age': {media_poblacional_age}")

Media poblacional de variable 'earn': 21147.3
Media poblacional de variable 'height': 66.57
Media poblacional de variable 'age': 42.93


In [51]:
# Se extrae algunas variables h0, h1 y resultado de pruebas realizadas para utilizarlas en función posterior
resultados_test_earn = [(pruebahipotesis['h0'], pruebahipotesis['h1'], pruebahipotesis['alpha'], pruebahipotesis['t_stat'],pruebahipotesis['resultado']) for pruebahipotesis in pruebas_earn]
resultados_test_height = [(pruebahipotesis['h0'], pruebahipotesis['h1'], pruebahipotesis['alpha'], pruebahipotesis['t_stat'],pruebahipotesis['resultado']) for pruebahipotesis in pruebas_height]
resultados_test_age = [(pruebahipotesis['h0'], pruebahipotesis['h1'], pruebahipotesis['alpha'], pruebahipotesis['t_stat'],pruebahipotesis['resultado']) for pruebahipotesis in pruebas_age]


In [63]:
def comparacion_parametros_estimadores(df, variable, resultados_test):
    media_poblacional = round(df[variable].mean(), 2)
    print(f"Media poblacional de la variable {variable}: {media_poblacional}")

    i = 1

    for h0, h1, alpha, t_stat, resultado in resultados_test:
        print(f"""********
Para la prueba de hipótesis '{variable}' número {i}, con:
        - H0 = {h0} y H1: {h1}
        - t-student: {t_stat}
        - {resultado}""")
        
        # Se calcula diferencia entre la media poblacional y H0
        diferencia = media_poblacional - h0
        print(f"La diferencia entre la media poblacional y H0 es de: {round(diferencia, 2)}")
        
        # Se calcula  el intervalo de confianza
        a, b = stats.t.interval(
            confidence=1-alpha,  # Nivel de confianza
            df=len(df[variable]) - 1,  # Grados de libertad
            loc=media_poblacional,  # Media poblacional
            scale=stats.sem(df[variable])  # Error estándar de la media
        )
        
        # Se calcula margen de error
        error = round(b - media_poblacional, 2)
        
        print(f"Con un nivel de significancia de {alpha}, y un margen de error de {error}, el intervalo de confianza va desde {round(a, 1)} hasta {round(b, 1)}.")

        if h0 < a or h0 > b:
            print(f"__________Valor de h0 está fuera de intervalo de confianza, por lo que se debería rechazar la hipótesis nula.\n")
        else:
            print(f"__________Valor de h0 está dentro del intervalo de confianza, por lo que se debería NO rechazar la hipótesis nula.\n")
        
        i += 1
        



In [64]:
comparacion_parametros_estimadores(df_earnings, 'earn', resultados_test_earn)

Media poblacional de la variable earn: 21147.3
********
Para la prueba de hipótesis 'earn' número 1, con:
        - H0 = 20000 y H1: Media poblacional diferente a 20000
        - t-student: -2.1380387085215293
        - Con nivel de confianza de 97.0%, NO se rechaza la hipótesis nula.
La diferencia entre la media poblacional y H0 es de: 1147.3
Con un nivel de significancia de 0.03, y un margen de error de 1148.3, el intervalo de confianza va desde 19999.0 hasta 22295.6.
__________Valor de h0 está dentro del intervalo de confianza, por lo que se debería NO rechazar la hipótesis nula.

********
Para la prueba de hipótesis 'earn' número 2, con:
        - H0 = 35000 y H1: Media poblacional diferente a 35000
        - t-student: -8.011771424240017
        - Con nivel de confianza de 95.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa.
La diferencia entre la media poblacional y H0 es de: -13852.7
Con un nivel de significancia de 0.05, y un margen de error de 1036.99, el

    Todos los resultados coinciden, menos la prueba número 3, en donde NO se rechaza h0, pero luego aparece que SÍ se debería rechazar.Veeamos qué ocurre con las otras variables.

In [65]:
comparacion_parametros_estimadores(df_earnings, 'height', resultados_test_height)

Media poblacional de la variable height: 66.57
********
Para la prueba de hipótesis 'height' número 1, con:
        - H0 = 55 y H1: Media poblacional diferente a 55
        - t-student: 17.64352905875293
        - Con nivel de confianza de 97.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa.
La diferencia entre la media poblacional y H0 es de: 11.57
Con un nivel de significancia de 0.03, y un margen de error de 0.2, el intervalo de confianza va desde 66.4 hasta 66.8.
__________Valor de h0 está fuera de intervalo de confianza, por lo que se debería rechazar la hipótesis nula.

********
Para la prueba de hipótesis 'height' número 2, con:
        - H0 = 68 y H1: Media poblacional diferente a 68
        - t-student: -1.860372111637205
        - Con nivel de confianza de 95.0%, NO se rechaza la hipótesis nula.
La diferencia entre la media poblacional y H0 es de: -1.43
Con un nivel de significancia de 0.05, y un margen de error de 0.18, el intervalo de confianza va desd

In [67]:
comparacion_parametros_estimadores(df_earnings, 'age', resultados_test_age)

Media poblacional de la variable age: 42.93
********
Para la prueba de hipótesis 'age' número 1, con:
        - H0 = 44 y H1: Media poblacional diferente a 44
        - t-student: -1.3023984977655982
        - Con nivel de confianza de 97.0%, NO se rechaza la hipótesis nula.
La diferencia entre la media poblacional y H0 es de: -1.07
Con un nivel de significancia de 0.03, y un margen de error de 0.87, el intervalo de confianza va desde 42.1 hasta 43.8.
__________Valor de h0 está fuera de intervalo de confianza, por lo que se debería rechazar la hipótesis nula.

********
Para la prueba de hipótesis 'age' número 2, con:
        - H0 = 56 y H1: Media poblacional diferente a 56
        - t-student: -4.354894976903721
        - Con nivel de confianza de 95.0%, se rechaza la hipótesis nula, a favor de una hipótesis alternativa.
La diferencia entre la media poblacional y H0 es de: -13.07
Con un nivel de significancia de 0.05, y un margen de error de 0.79, el intervalo de confianza va desde 42.

    En esta última, no coincide ni la prueba 1 ni la 3

-
-
-
___0___o___
-
-
-

#### 2. Crea una función que estime la proporción de hombres en el dataset, considerando una muestra de 50 individuos, con niveles de significancia de 0,05 y 0,01. Escribe con palabras tus resultados ejecutar la función e interpreta. Compara con la proporción real.

In [22]:
def estimar_proporcion_hombres(data, n=50, alpha=0.05):
    # Tomar una muestra de tamaño n
    muestra = data['male'].sample(n=n, random_state=1)
    
    # Calcular la proporción de hombres en la muestra
    proporcion_muestra = muestra.mean()
    
    # Calcular la proporción real de hombres en la población
    proporcion_real = data['male'].mean()
    
    # Calcular el estadístico z
    p_hat = proporcion_muestra
    p = proporcion_real
    z = (p_hat - p) / np.sqrt((p * (1 - p)) / n)
    
    # Calcular el valor crítico
    z_critico = stats.norm.ppf(1 - alpha/2)
    
    # Determinar si se rechaza o no la hipótesis nula
    if abs(z) > z_critico:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se rechaza la hipótesis nula"
    else:
        resultado = f"Con nivel de confianza del {100*(1-alpha)}%, se acepta la hipótesis nula"
    
    return proporcion_muestra, proporcion_real, resultado

# Aplicar la función con niveles de significancia de 0.05 y 0.01
proporcion_muestra_005, proporcion_real, resultado_005 = estimar_proporcion_hombres(df_earnings, alpha=0.05)
proporcion_muestra_001, _, resultado_001 = estimar_proporcion_hombres(df_earnings, alpha=0.01)

# Mostrar los resultados
print(f"Proporción de hombres en la muestra (alpha=0.05): {proporcion_muestra_005}")
print(f"Proporción real de hombres en la población: {proporcion_real}")
print(f"Resultado (alpha=0.05): {resultado_005}")

print(f"Proporción de hombres en la muestra (alpha=0.01): {proporcion_muestra_001}")
print(f"Resultado (alpha=0.01): {resultado_001}")

Proporción de hombres en la muestra (alpha=0.05): 0.3
Proporción real de hombres en la población: 0.3716960352422907
Resultado (alpha=0.05): Con nivel de confianza del 95.0%, se acepta la hipótesis nula
Proporción de hombres en la muestra (alpha=0.01): 0.3
Resultado (alpha=0.01): Con nivel de confianza del 99.0%, se acepta la hipótesis nula


-
-
-
___0___o___
-
-
-

#### 3. Considerando el dataset como una muestra, verifica si el género (male=1 significa ‘hombre”) influye sobre el sueldo “earn” de las personas. Explica y justifica tu procedimiento.

In [72]:
# Se separan las variables independientes en dos grupos: hombres y mujeres
sueldo_hombres = df_earnings[df_earnings['male'] == 1]['earn']
sueldo_mujeres = df_earnings[df_earnings['male'] == 0]['earn']


In [81]:
sueldo_hombres.describe()

count       675.000000
mean      30105.179259
std       28851.517976
min           0.000000
25%       15000.000000
50%       25000.000000
75%       36000.000000
max      400000.000000
Name: earn, dtype: float64

In [82]:
sueldo_mujeres.describe()

count      1141.000000
mean      15847.935145
std       15508.575961
min           0.000000
25%        5000.000000
50%       15000.000000
75%       24000.000000
max      136500.000000
Name: earn, dtype: float64

In [77]:
# Se realiza una prueba t para muestras independientes
t_stat, p_value = stats.ttest_ind(sueldo_hombres, sueldo_mujeres, equal_var=False)
print(t_stat)
print(p_value)

11.86459381795286
2.7373648004575493e-30


In [75]:
# Nivel de significancia
alpha = 0.05

In [78]:
# Se interpretan los resultados
if p_value < alpha:
    resultado = f"Con un nivel de significancia de {alpha}, se rechaza la hipótesis nula. Hay una diferencia significativa en los sueldos entre hombres y mujeres."
else:
    resultado = f"Con un nivel de significancia de {alpha}, no se rechaza la hipótesis nula. No hay una diferencia significativa en los sueldos entre hombres y mujeres."

In [80]:
# Se muestran los resultados
print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")
print(resultado)


Estadístico t: 11.86459381795286
Valor p: 2.7373648004575493e-30
Con un nivel de significancia de 0.05, se rechaza la hipótesis nula. Hay una diferencia significativa en los sueldos entre hombres y mujeres.


Explicación y justificación del procedimiento

Para verificar si el género influye sobre el sueldo, se realizó una prueba t para muestras independientes.
Esta prueba compara las medias de dos grupos independientes (hombres y mujeres) para determinar si hay una diferencia significativa entre ellas.

Pasos realizados:
1. Se separaron los datos en dos grupos: hombres y mujeres.
2. Se realizó la prueba t para muestras independientes, asumiendo varianzas desiguales (equal_var=False).
3. Se calculó el estadístico t y el valor p.
4. Se comparó el valor p con el nivel de significancia (alpha = 0.05) para determinar si se rechaza o no la hipótesis nula.

Esto dio como ressultado:
con un nivel de significancia de 0.05, se rechaza la hipótesis nula. Esto indica que hay una diferencia significativa en los sueldos entre hombres y mujeres.

-
-
-
___0_FiN__o___
-
-
-

### Requerimientos

1. Realiza pruebas de hipótesis, considerando el enunciado de cada una y las implementa en Python mediante funciones (5 Puntos)

2. Infiere sobre medias poblacionales y proporciones, interpretando error y significancia. (2 Puntos)

3. Plantea, interpreta e implementa hipótesis de prueba para muestras independientes. (3 Puntos)